# Advent of Code - 2025 - Day 10 - Problem 2

https://adventofcode.com/2025/day/10

Hint provided by https://www.reddit.com/r/adventofcode/comments/1pk87hl/2025_day_10_part_2_bifurcate_your_way_to_victory/

## Load Source Data

Load source data into `DATA`.

In [1]:
# Read the input file containing turn instructions
with open("data/day10.txt") as f:
    DATA = [line.strip() for line in f]

# DATA

## Define Machine class

Defines the state of a machine.

In [2]:
import itertools
from functools import cache
import re


class Machine:

    def __init__(self, definition: str):

        valid_lights, buttons, joltages = Machine.parse_definition(definition)

        self.valid_lights = valid_lights
        self.buttons = buttons
        self.joltages = joltages

    @staticmethod
    def create_patterns() -> tuple[str, str]:

        ws = r"\s+"
        light_pattern = r"\[([.#]+)\]"
        button_pattern = r"\(([0-9,]+)\)"
        joltage_pattern = r"\{([0-9,]+)\}"

        # Regular expression pattern for the definition string
        line_pattern = (
            f"{light_pattern}((?:{ws}{button_pattern})+){ws}{joltage_pattern}"
        )

        # Regular expression for individual button definitions
        button_pattern = f"(?:{ws}({button_pattern}))"

        return (line_pattern, button_pattern)

    @staticmethod
    def parse_definition(
        definition: str,
    ) -> tuple[list[bool], list[list[int]], list[int]]:

        # Parse the overall definition string. Note that there are two results returned for the buttons due to the nested grouping constructs: the entire
        # button string as well as the last matched button.
        #
        definition_match = re.match(Machine.line_pattern, definition)
        assert definition_match
        buttons = definition_match.group(2)
        button_matches = re.findall(Machine.button_pattern, buttons)

        # Parse and convert the separate machine attributes.
        #
        valid_lights: list[bool] = [light == "#" for light in definition_match.group(1)]
        buttons = [
            list(map(int, button_match[1].split(",")))
            for button_match in button_matches
        ]
        joltages = list(map(int, definition_match.group(4).split(",")))

        return (valid_lights, buttons, joltages)

    @cache
    def create_joltage_primitives(self) -> dict[tuple[int, ...], int]:
        """
        Creates a dictionary of joltage primitives. Created by trying each unique combination of button presses. The value
        is the minimum number of button presses (when there is more than one possible combination.)
        """

        result: dict[tuple[int, ...], int] = dict()

        for set_size in range(1, len(self.buttons) + 1):
            for button_set in itertools.combinations(self.buttons, set_size):
                button_count = 0
                joltages = [0] * len(self.joltages)
                for button in button_set:
                    button_count += 1
                    for idx_joltage in button:
                        joltages[idx_joltage] += 1
                key = tuple(joltages)
                # print(f"button_set={button_set}, joltages={joltages}, button_count={button_count}")
                if key in result:
                    result[key] = min(button_count, result[key])
                else:
                    result[key] = button_count

        result[tuple([0] * len(self.joltages))] = 0
        return result

    def get_cost(
        self,
        joltages: tuple[int, ...],
        primitives: dict[tuple[int, ...], int],
        level: int,
    ) -> int:

        if all(j == 0 for j in joltages):
            return 0

        result = 999999999

        for primitive, primitive_cost in primitives.items():
            if all(p <= j and p % 2 == j % 2 for p, j in zip(primitive, joltages)):
                remaining_joltages = tuple(
                    (j - p) // 2 for p, j in zip(primitive, joltages)
                )
                remaining_cost = 2 * self.get_cost(
                    remaining_joltages, primitives, level + 1
                )
                result = min(result, primitive_cost + remaining_cost)

        return result

    line_pattern, button_pattern = create_patterns()

## Solve All Machines

In [3]:
total_cost = 0

for idx, line in enumerate(DATA):
    print(f"============ {idx}")
    m = Machine(line)
    primitives = m.create_joltage_primitives()
    cost = m.get_cost(tuple(m.joltages), primitives, 0)
    total_cost += cost

print(f"total_cost = {total_cost}")

============ 0
============ 1
============ 2
============ 3
============ 4
============ 5
============ 6
============ 7
============ 8
============ 9
============ 10
============ 11
============ 12
============ 13
============ 14
============ 15
============ 16
============ 17
============ 18
============ 19
============ 20
============ 21
============ 22
============ 23
============ 24
============ 25
============ 26
============ 27
============ 28
============ 29
============ 30
============ 31
============ 32
============ 33
============ 34
============ 35
============ 36
============ 37
============ 38
============ 39
============ 40
============ 41
============ 42
============ 43
============ 44
============ 45
============ 46
============ 47
============ 48
============ 49
============ 50
============ 51
============ 52
============ 53
============ 54
============ 55
============ 56
============ 57
============ 58
============ 59
============ 60
============ 61
============ 62
==